<a href="https://colab.research.google.com/github/MekaRaptor/Graduation_Project/blob/main/Model2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import DataLoader, Dataset
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os

drive_path = "/content/drive/MyDrive/"  # ZIP dosyanın bulunduğu klasör
os.listdir(drive_path)  # Klasördeki dosyaları listele


['Classroom',
 'Alper_Ağırman_201180010 (1).txt',
 'Alper_Ağırman_201180010.txt',
 'Ceng307_201180010_HW1.sln',
 'Bilge Adam Data Science',
 'Untitled Diagram (4).drawio',
 'Untitled Diagram (3).drawio',
 'Document (1).sdr',
 'Untitled Diagram (2).drawio',
 'AI_Model - Made with Clipchamp_1718179352783.mp4',
 'Project Details',
 'Adsız sunu.gslides',
 'Untitled Diagram (1).drawio',
 'Neural_Network Documentation dokümanının çevrilmiş kopyası.gdoc',
 'Neural_Network Documentation.gdoc',
 'Literatür Taraması.gdoc',
 'Adsız doküman (2).gdoc',
 'Staj Raporu.gdoc',
 'Neural_Network Documentation (1).pdf',
 'staj2018-sablon (3).docx',
 'SRS_aciklamalar.gdoc',
 'SRS-Sablon.gdoc',
 'SDD_aciklamalar (1).gdoc',
 'SDD_aciklamalar.gdoc',
 'Untitled Diagram.drawio',
 'SPMP_Ornek.gdoc',
 'SDD-Sablon.gdoc',
 'week9.gdoc',
 'llm.gdoc',
 'clustering.gdoc',
 'cnn_rnn_gnn.gdoc',
 'NN.gdoc',
 'GAZI Midterm.gdoc',
 'Meet Recordings',
 'Bilgilendirme Metni (1).docx',
 'Bilgilendirme Metni.docx',
 'Do

In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/Bitirme/Agriculture-Vision-2021.zip"  # ZIP dosyasının gerçek adını yaz
extract_path = "/content/veri_seti"  # Açılacağı yer

# ZIP dosyasını açma işlemi
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ ZIP dosyası başarıyla çıkarıldı.")


✅ ZIP dosyası başarıyla çıkarıldı.


In [ ]:
dataset_path = "/content/veri_seti/Agriculture-Vision-2021"

# Ana klasörleri listeleyelim
folders = os.listdir(dataset_path)
print("📂 Veri Seti İçeriği:")
for folder in folders:
    print("-", folder)


📂 Veri Seti İçeriği:
- train
- val
- test


In [ ]:
import os

# Ana klasör
dataset_path = "/content/veri_seti/Agriculture-Vision-2021"
train_path = os.path.join(dataset_path, "train")

# Alt klasör yolları
rgb_path = os.path.join(train_path, "images", "rgb")
nir_path = os.path.join(train_path, "images", "nir")
mask_path = os.path.join(train_path, "masks")  # ⬅️ Eksik olan burasıydı
label_path = os.path.join(train_path, "labels")

# Etiket sınıflarını klasör olarak alalım
label_classes = os.listdir(label_path)

print("📁 Etiket Sınıfları:")
for cls in label_classes:
    print("-", cls)


📁 Etiket Sınıfları:
- water
- waterway
- double_plant
- nutrient_deficiency
- endrow
- drydown
- planter_skip
- weed_cluster
- storm_damage


In [ ]:
# Custom Dataset
class AgricultureVisionDataset(Dataset):
    def __init__(self, image_dir, label_dir, file_list, transform=None, label_classes=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.file_list = file_list
        self.transform = transform
        self.label_classes = label_classes or []

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        base_name = os.path.splitext(file_name)[0]
        img_path = os.path.join(self.image_dir, file_name)

        # RGB görüntüyü oku
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Çok sınıflı maske oluştur
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        for class_index, cls in enumerate(self.label_classes, 1):
            label_path = os.path.join(label_path_global, cls, base_name + ".png")
            if os.path.exists(label_path):
                class_mask = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
                if class_mask is not None:
                    mask[class_mask > 0] = class_index - 1

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

# 🌿 Train ve Validation için Transformlar
train_transform = A.Compose([
    A.Resize(512, 512),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

print("✅ Dataset ve Transformasyonlar hazır!")


✅ Dataset ve Transformasyonlar hazır!


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"📍 Kullanılan cihaz: {device}")


📍 Kullanılan cihaz: cuda


In [ ]:
import torchvision.models.segmentation as models
import torch.nn as nn

# DeepLabV3+ modelini yükle
model = models.deeplabv3_resnet50(pretrained=True)

# Çıkış katmanını (classifier) tarımsal sınıflarımıza göre yeniden tanımlıyoruz (9 sınıf)
model.classifier[4] = nn.Conv2d(256, 9, kernel_size=(1, 1))

# Cihaza taşı
model = model.to(device)

print("✅ DeepLabV3+ (ResNet-50 backbone) modeli başarıyla hazırlandı.")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 211MB/s]


✅ DeepLabV3+ (ResNet-50 backbone) modeli başarıyla hazırlandı.


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


In [ ]:


# ✅ Maskesi düzgün olan görüntülerin dosya isimlerini listele
valid_filenames = []

for mask_file in os.listdir(mask_path):
    mask_full_path = os.path.join(mask_path, mask_file)
    mask = cv2.imread(mask_full_path, cv2.IMREAD_GRAYSCALE)

    if mask is not None and np.any(mask > 0):
        valid_filenames.append(mask_file.replace(".png", ".jpg"))  # .png yerine .jpg olacak

print(f"✅ Geçerli örnek sayısı: {len(valid_filenames)}")
print("🔹 İlk 5 örnek:", valid_filenames[:5])


✅ Geçerli örnek sayısı: 56944
🔹 İlk 5 örnek: ['8EF9M4P6Y_20842-10557-21354-11069.jpg', '98BMWB9B8_11365-11777-11877-12289.jpg', 'GTLUAY7IY_8424-9211-8936-9723.jpg', 'NL8XF3W12_3485-3832-3997-4344.jpg', 'YY8EPIW1C_9693-15330-10205-15842.jpg']


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

# 🔥 Eğitim için uygun augmentasyonlar
transform = A.Compose([
    A.Resize(512, 512),  # Görüntüleri modelin beklediği boyuta getiriyoruz
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


In [ ]:
# 📂 Agriculture-Vision veri setindeki 9 sınıf
class_names = [
    'nutrient_deficiency',
    'storm_damage',
    'planter_skip',
    'endrow',
    'waterway',
    'water',
    'drydown',
    'weed_cluster',
    'double_plant'
]


In [ ]:
from torch.utils.data import DataLoader

train_dataset = AgricultureVisionDataset(
    image_dir=rgb_path,
    label_dir=label_path,
    file_list=valid_filenames,
    transform=transform,
    label_classes=class_names
)


In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=4,   # İstersen artırabiliriz
    shuffle=True,
    num_workers=2,  # Colab için ideal
    pin_memory=True  # GPU kullanımı için ek performans sağlar
)

print(f"✅ Train DataLoader başarıyla oluşturuldu. Toplam batch sayısı: {len(train_loader)}")


✅ Train DataLoader başarıyla oluşturuldu. Toplam batch sayısı: 14236


In [ ]:
import torch.optim as optim

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# 🔥 Learning Rate Scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
# 🔁 Dataset içi kullanmak için global değişken
label_path_global = label_path


In [ ]:
epochs = 10
model.train()

for epoch in range(epochs):
    total_loss = 0.0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device).long()

        optimizer.zero_grad()
        outputs = model(images)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📈 Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

    # 🔥 Öğrenme oranını güncelle
    scheduler.step()


📈 Epoch 1/10, Loss: 0.6177
📈 Epoch 2/10, Loss: 0.4912
📈 Epoch 3/10, Loss: 0.4459
📈 Epoch 4/10, Loss: 0.4166
📈 Epoch 5/10, Loss: 0.3938
📈 Epoch 6/10, Loss: 0.3547
📈 Epoch 7/10, Loss: 0.3395
📈 Epoch 8/10, Loss: 0.3300
📈 Epoch 9/10, Loss: 0.3225
📈 Epoch 10/10, Loss: 0.3140


In [ ]:
torch.save(model.state_dict(), "deeplabv3_agriculture_rgb21.pth")
print("✅ Model başarıyla kaydedildi.")


✅ Model başarıyla kaydedildi.


In [ ]:
# 📦 Gerekli kütüphaneler


# 🧩 Test Dataset sınıfı
class TestAgricultureVisionDataset(Dataset):
    def __init__(self, image_dir, label_dir, file_list, transform=None, label_classes=None):
        self.image_dir = image_dir
        self.label_dir = label_dir
        self.file_list = file_list
        self.transform = transform
        self.label_classes = label_classes or []

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_name = self.file_list[idx]
        base_name = os.path.splitext(file_name)[0]
        img_path = os.path.join(self.image_dir, file_name)

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
        for class_index, cls in enumerate(self.label_classes, 1):
            label_path = os.path.join(label_path_global, cls, base_name + ".png")
            if os.path.exists(label_path):
                class_mask = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE)
                if class_mask is not None:
                    mask[class_mask > 0] = class_index - 1

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = torch.tensor(augmented['mask'], dtype=torch.long)

        return image, mask


In [ ]:
test_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'valid_filenames' contains all your image filenames
train_files, val_files = train_test_split(valid_filenames, test_size=0.2, random_state=42)

In [ ]:
# 💿 Test Dataset oluştur
val_dataset = TestAgricultureVisionDataset(
    image_dir=rgb_path,
    label_dir=label_path,
    file_list=val_files,         # daha önce train_test_split yaptığımız
    transform=test_transform,
    label_classes=class_names
)

val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)


In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Sınıf sayısı
num_classes = 9

# Karışıklık matrisi başlat
total_conf_matrix = np.zeros((num_classes, num_classes), dtype=np.int64)

model.eval()
with torch.no_grad():
    for images, masks in val_loader:  # val_loader hazır olmalı
        images = images.to(device)
        outputs = model(images)['out']
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        targets = masks.cpu().numpy()

        for p, t in zip(preds, targets):
            cm = confusion_matrix(t.flatten(), p.flatten(), labels=list(range(num_classes)))
            total_conf_matrix += cm

# IoU hesaplama
intersection = np.diag(total_conf_matrix)
ground_truth_set = total_conf_matrix.sum(axis=1)
predicted_set = total_conf_matrix.sum(axis=0)
union = ground_truth_set + predicted_set - intersection

iou_per_class = intersection / (union + 1e-6)
mean_iou = np.mean(iou_per_class)

# Sonuçları yazdır
for i, iou in enumerate(iou_per_class):
    print(f"Sınıf {i}: IoU = {iou:.4f}")

print(f"\n📊 Ortalama IoU (Mean IoU): {mean_iou:.4f}")


Streaming output truncated to the last 5000 lines.
<ipython-input-24-038bc1ed99bd>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(augmented['mask'], dtype=torch.long)
<ipython-input-24-038bc1ed99bd>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(augmented['mask'], dtype=torch.long)
<ipython-input-24-038bc1ed99bd>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(augmented['mask'], dtype=torch.long)
<ipython-input-24-038bc1ed99bd>:35: UserWarning: To copy construct from a te

Sınıf 0: IoU = 0.8251
Sınıf 1: IoU = 0.3876
Sınıf 2: IoU = 0.2982
Sınıf 3: IoU = 0.1764
Sınıf 4: IoU = 0.3286
Sınıf 5: IoU = 0.2927
Sınıf 6: IoU = 0.4968
Sınıf 7: IoU = 0.4185
Sınıf 8: IoU = 0.2666

📊 Ortalama IoU (Mean IoU): 0.3878


<ipython-input-24-038bc1ed99bd>:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask = torch.tensor(augmented['mask'], dtype=torch.long)
